# Mind The Gap: Domain Gap Contol For Single Shot Domain Adaptation For Generative Adversarial Networks


In [10]:
#@title Setup. This will take a few minutes.
import torch
torch.backends.cudnn.benchmark = True

# from google.colab import files
from copy import deepcopy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from google.colab import auth
from oauth2client.client import GoogleCredentials


In [12]:
#@title Download models
#@markdown You may optionally enable downloads with pydrive in order to authenticate and avoid drive download limits.
download_with_pydrive = True #@param {type:"boolean"}    
device = 'cuda' #@param ['cuda', 'cpu']

drive_ids = {
    'e4e_ffhq_encode.pt': '1O8ipkyMYHwCRmuaZBaO-KYZ9FYuH8Xnc',
    'ffhq.pt': '1H9-wdGHMu0E6H-QXOmRXLZVy_E_D0mf0',
    'titan_erwin.pt': '1AvSrWkIxgoxXtjPuogKiP45BkKJvFQbI',
    'titan_armin.pt': '1o9yhTmW8voeCi6dNrOY3sSVQqMawftRB',
    'titan_historia.pt': '1MqMmdcCGXutoDV8wxP31K2iyvA4SgJx7',
    'pocahontas.pt': '1jRcWh7lQ-28abiSOVBVUi_iBDGPj7Esl',
    'moana.pt': '19kjijHa_G2B3UrNGHXYVe5izaNdi8ABr',
    'doc_brown.pt': '1fQJYUE9a9DSoRupOxllslA-dIm-8-D72',
    'brave.pt': '1wD3xoGgrmbN74npUAlrmkk97_DXF6NWR',
    'sketch.pt': '1YbFyukh6n9l6UFtsqOESbhwGH7BupJtb',
    'jojo.pt': '1VLLmh7f-vcS2MB3CXET1lFRC-_Sq-p8J',
    'detroit.pt': '1cYSX9oLkhv6vosIAKnEQZoDFQS5siT-u',
    'picasso.pt': '1C7pCKIFdqFFrK9diFZKGLXrYlmrPP2Ui',
    'anastasia.pt': '1iXxaxKG0EJ_C1Jr5QrBogq9yvMzyhcth',
    'room_girl.pt': '16F1oCrv8UNnhlqFFUwJy49OpW5d-kA-m',
    'speed_paint.pt': '1uB2uQnAF8pghXNlTnGRTduiwpAtcDiqo',
    'digital_painting_jing.pt': '168bfp7FvN_VF1pOT7uEYzhtj2oFGIsQ_',
    'mermaid.pt': '1LO3UdMHPKfwjaaxgVgBbLquei8n8q5P8',
    'zbrush_girl.pt': '1YPQSDW-_utOEu5A9nbq2832jTTJK2ECb',
    'joker.pt': '1Ptv-EjYAKngxpf9lY5cpypAmyoX863Nh',
}

# from StyelGAN-NADA
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
#         auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_name):
        file_dst = os.path.join('models', file_name)
        file_id = drive_ids[file_name]
        if not os.path.exists(file_dst):
            print(f'Downloading {file_name}')
            if self.use_pydrive:
                downloaded = self.drive.CreateFile({'id':file_id})
                downloaded.FetchMetadata(fetch_all=True)
                downloaded.GetContentFile(file_dst)
            else:
                !gdown --id $file_id -O $file_dst



downloader = Downloader(download_with_pydrive)

downloader.download_file('ffhq.pt')
downloader.download_file('e4e_ffhq_encode.pt')

latent_dim = 512

# Load original generator
original_generator = Generator(1024, latent_dim, 8, 2).to(device)
ckpt = torch.load('models/ffhq.pt', map_location=lambda storage, loc: storage)
original_generator.load_state_dict(ckpt["g_ema"])
mean_latent = original_generator.mean_latent(10000)

# to be finetuned generator
generator = deepcopy(original_generator)


transform = transforms.Compose(
    [
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

ApplicationDefaultCredentialsError: The Application Default Credentials are not available. They are available if running in Google Compute Engine. Otherwise, the environment variable GOOGLE_APPLICATION_CREDENTIALS must be defined pointing to a file defining the credentials. See https://developers.google.com/accounts/docs/application-default-credentials for more information.